## S&P 500 Companies Database Insert Script

### Description:
This Python script connects to a PostgreSQL database and inserts S&P 500 company data from a DataFrame into a pre-existing table. The script includes error handling and uses a conflict resolution strategy to avoid duplicate entries based on the ticker symbol.

### Database Connection Details:
- **Host**: npatelnynj.ddns.net
- **Port**: 5432
- **Database**: postgres
- **Username**: postgres
- **Password**: Aarav12#$

### Table Definition:
```sql
CREATE TABLE companies (
    company_id SERIAL PRIMARY KEY
    ticker VARCHAR(20) UNIQUE NOT NULL,
    company_name VARCHAR(255) NOT NULL,
    sector VARCHAR(100),
    industry VARCHAR(100),
    headquarters VARCHAR(255),    
    added_date DATE NOT NULL
);

In [ ]:
import pandas as pd

# Wikipedia URL for S&P 500 companies
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Function to fetch S&P 500 company data from Wikipedia
def fetch_sp500_companies():
    tables = pd.read_html(URL)
    df = tables[0]  # Extract first table

    # Print available columns to verify
    print("Available Columns:", df.columns.tolist())

    # Adjusting for possible column name changes
    column_map = {
        'Symbol': 'ticker',
        'Security': 'company_name',
        'GICS Sector': 'sector',
        'GICS Sub-Industry': 'industry',
        'Date added': 'added_date',  # Fixed: 'Date first added' may have changed to 'Date added'
        'Headquarters Location': 'headquarters'
    }

    # Keep only available columns
    available_columns = {col: new_col for col, new_col in column_map.items() if col in df.columns}
    df = df[list(available_columns.keys())]
    df.rename(columns=available_columns, inplace=True)

    # Convert "added_date" to DATE format
    if 'added_date' in df.columns:
        df['added_date'] = pd.to_datetime(df['added_date'], errors='coerce').dt.date

    # Print the first few rows
    print(df.head())

    return df

# Fetch and print the data
sp500_df = fetch_sp500_companies()

In [4]:
import psycopg2

# Database connection parameters
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "Aarav12#$",
    "host": "npatelnynj.ddns.net",
    "port": "5432"
}

# Function to insert data into PostgreSQL
def insert_sp500_data(df):
    try:
        conn = psycopg2.connect(**DB_PARAMS)
        cur = conn.cursor()

        # Insert query
        INSERT_QUERY = """
        INSERT INTO companies (ticker, company_name, sector, industry, headquarters,added_date)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (ticker) DO NOTHING;
        """

        # Insert each row into PostgreSQL using .itertuples() for efficiency
        for row in df.itertuples(index=False):
            cur.execute(INSERT_QUERY, (row.ticker, row.company_name, row.sector, row.industry, row.headquarters, row.added_date))

        # Commit and close connection
        conn.commit()
        print("✅ S&P 500 company data inserted successfully!")

    except Exception as e:
        print("❌ Database error:", e)

    finally:
        cur.close()
        conn.close()

# Insert the fetched data into your existing table
insert_sp500_data(sp500_df)

✅ S&P 500 company data inserted successfully!
